In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import distutils

In [2]:
train_datagen = ImageDataGenerator( rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

train_generator = train_datagen.flow_from_directory(directory='../../Data/motion detector/train',
                                                   target_size = (64,64),
                                                   color_mode='rgb',
                                                   batch_size = 16,
                                                   class_mode = 'categorical',
                                                   shuffle = True,
                                                   seed = 42)



Found 376 images belonging to 3 classes.


In [3]:
valid_datagen = ImageDataGenerator( rescale = 1./255)

validation_generator = valid_datagen.flow_from_directory(directory='../../Data/motion detector/validation',
                                                   target_size = (64,64),
                                                   color_mode='rgb',
                                                   batch_size = 16,
                                                   class_mode = 'categorical',
                                                   shuffle = True,
                                                   seed = 42)



Found 84 images belonging to 3 classes.


In [4]:
def create_model():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.Input(shape=(64,64,3)))
    model.add(tf.keras.layers.Conv2D(6, (3,3), padding='same', activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    model.add(tf.keras.layers.Conv2D(16, (3,3), padding='same', activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    model.add(tf.keras.layers.Conv2D(32, (3,3), padding='same', activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    model.add(tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    model.add(tf.keras.layers.Dropout(0.25))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(512))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(128))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(64))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(3))
    model.add(tf.keras.layers.Activation('softmax'))
    return model

In [5]:
model = create_model()
model.compile( optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
             loss='categorical_crossentropy',
             metrics=['categorical_accuracy'])

model.fit(train_generator,
         epochs=10,
         validation_data=validation_generator,
         validation_freq=1)

Epoch 1/10
24/24 [==============================] - 13s 548ms/step - loss: 0.9665 - categorical_accuracy: 0.4840 - val_loss: 0.5375 - val_categorical_accuracy: 0.8452
Epoch 2/10
24/24 [==============================] - 7s 292ms/step - loss: 0.5608 - categorical_accuracy: 0.7872 - val_loss: 0.2473 - val_categorical_accuracy: 0.8690
Epoch 3/10
24/24 [==============================] - 6s 268ms/step - loss: 0.4240 - categorical_accuracy: 0.8324 - val_loss: 0.2516 - val_categorical_accuracy: 0.9286
Epoch 4/10
24/24 [==============================] - 7s 278ms/step - loss: 0.3914 - categorical_accuracy: 0.8644 - val_loss: 0.2319 - val_categorical_accuracy: 0.9405
Epoch 5/10
24/24 [==============================] - 7s 273ms/step - loss: 0.3423 - categorical_accuracy: 0.8777 - val_loss: 0.2251 - val_categorical_accuracy: 0.9405
Epoch 6/10
24/24 [==============================] - 7s 274ms/step - loss: 0.3456 - categorical_accuracy: 0.8750 - val_loss: 0.2033 - val_categorical_accuracy: 0.9405
Epo

In [6]:
model.save('model.h5', overwrite=True)


In [7]:
import cv2
import numpy as np

In [15]:
cap = cv2.VideoCapture('../../Data/motion detector/test_video.mov')

fourcc = cv2.VideoWriter_fourcc('M', 'J', 'P', 'G')
out = cv2.VideoWriter('output_count.avi',fourcc,20.0,(int(cap.get(3)), int(cap.get(4))))

ret, frame1 =cap.read()
# cvtColor() method is used to convert an image from one color space to another
# we use the function cv2. cvtColor(input_image, flag) where flag determines the type of conversion. For BGR Gray conversion we use the flags cv2.COLOR_BGR2GRAY

prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
# Create mask
hsv = np.zeros_like(frame1)
# Set image saturation to maximum value as we do not need it
hsv[...,1] = 255

i=0
prediction_str = ""
repetitions=0
up=0
down=0
no_move=0
current_move=0
initial =-1

while(cap.isOpened()):
    i+=1
    # Read a frame from video
    ret, frame2 = cap.read()
    if not(ret):
        break
    # Convert new frame format`s to gray scale
    next = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
    # Calculate dense optical flow by Farneback method
    flow = cv2.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    # Compute the magnitude and angle of the 2D vectors
    mag, ang = cv2.cartToPolar(flow[...,0],flow[...,1])
    # Set image hue according to the optical flow direction
    hsv[...,0] = ang*180/np.pi/2
    # Set image value according to the optical flow magnitude (normalized)
    hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    # Convert HSV to RGB (BGR) color representation
    rgb = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
    # Resize frame size to match dimensions
    image = cv2.resize(rgb, (64, 64))
    image = image.reshape((1,) + image.shape)
    image = image/255.0
    prediction = np.argmax(model.predict(image), axis=-1)[0]
    
    if prediction == 0:
        down +=1 
        if down == 3:
          if initial == -1:
            initial = 0
          if current_move == 2:
            repetitions+=1
          current_move = 0
        elif down > 0:
          up = 0
          no_move = 0
    elif prediction == 2:
        up += 1
        if up == 3 and initial != -1:
          current_move = 2
        elif up > 1:
          down = 0 
          no_move = 0
    else:
        no_move += 1
        if no_move == 15:
          current_move = 1
        elif no_move > 10:
          up = 0
          down = 0 
    font                   = cv2.FONT_HERSHEY_SIMPLEX
    bottomLeftCornerOfText = (10,400)
    fontScale              = 1
    fontColor              = (255,255,255)
    lineType               = 5
    cv2.putText(frame2, "Repetitions: "+ str(repetitions),bottomLeftCornerOfText,font, fontScale,fontColor,lineType)
    out.write(frame2)
    prvs = next

print("Video Generated")
out.release()
cap.release()
cv2.destroyAllWindows()


Video Generated
